In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.streaming import StreamingContext
from Activity import Activity
from activityByProduct import activityByProduct

In [2]:
from pyspark.streaming.kafka import KafkaUtils

In [3]:
sc = SparkContext.getOrCreate()
sqlcontext = SQLContext(sc)

In [4]:
ssc = StreamingContext(sc,4)

In [5]:
# inputPath = "/home/shravan/Documents/big_data/data_engg/Lambda-architecture/stream_data"
# textDStream = ssc.textFileStream(inputPath)
textDStream = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'data_pipeline':1})

In [6]:
# lines = textDStream.map(lambda x: x[1])
# print(type(lines))
# # lines.
# lines.pprint()
# ssc.start()  

In [7]:
def subprocess(line):
    print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
    print(type(line))    
    records = ''.join(line).split("\t")
    if records[0] != "action" and len(records) == 7:
        return Activity(records[0],records[1],records[2],records[3],records[4],records[5],records[6])
    else:
        return None

In [8]:
def process(rdd):
    newRDD = rdd.map(subprocess)
    return newRDD

In [9]:
def process_subquery(line):
    return (line[0],line[1]),activityByProduct(line[0],line[1],line[2],line[3],line[4])

In [10]:
def process_query(rdd):
    print("Inside foreach")
    newDF = rdd.toDF()
    newDF.write.mode("append").parquet("hdfs://localhost:9000/kafka_spark")
    newDF.createOrReplaceTempView("activity")
    activityByProduct = sqlcontext.sql("SELECT product, timestamp_hour, sum(case when action = 'purchase' then 1 else 0 end) as purchase_count, sum(case when action = 'add_to_cart' then 1 else 0 end) as add_to_cart_count, sum(case when action = 'page_view' then 1 else 0 end) as page_view_count from activity group by product, timestamp_hour")
    #activityByProduct.createOrReplaceTempView("activityByProduct")
    activityByProduct1 = activityByProduct.rdd
    finalRDD =activityByProduct1.map(process_subquery)
    return finalRDD 

In [11]:
#records = textDStream.map(process)
#records.pprint()
newRdd = textDStream.transform(process)
#newRdd.pprint()


In [12]:
transRDD = newRdd.transform(process_query)
transRDD.pprint()

In [13]:
#textDStream.pprint()

In [14]:
ssc.start()

Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside for

Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside for

Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside foreach
Inside for

In [ ]:
ssc.awaitTermination()